# winter 2022 week03

## goals:

reload the data
run util/load_data.py convert_all() function -- which loads everything and converts it to parquet

## todo
- ~add PIPA to list of interesting ones (57 -> 58)~
- ~figure out how to get the thing to run faster (try to just use geopoints_sampled in analyze)~
- ~final count of reduced fishing~
- run the analyze thing over those 7/8
- make those charts of in/out fishing effort of individual ships for those 7/8
- make pictures of before/after fishing effort of individual ships for those 7/8 (could use consistent colors)

THEN:
- ~make ~table~ scatter-plot of (low, low), (low, high), (high, low), (high, high). etc. of the 171~
- ~make table of (?, low), (?, high) of all the other ones~
- try to categorize the mpas (sum up area and #, etc.)

### of the 171:
- about 14 showed notable decrease in both # hours and %
- significant number (>100) of the 171 had no effort in before or after
- very small number (3) showed a modest increase in % internal effort
(should I quantify this by marine area?)
- any sort of before/after pictures helpful? (at least make the before/after effort plots)
- decide on criteria: % decrease by at least something
- for all the matching ones, do a before/after



AND THEN:
- compare to the paper about 5 MPAs
- email Dan to show him everything
- title: what can GFW data tell us about the true protection of MPA?
- two effective scenarios: (high, low) and (low, low) ~= (?, low)


## notes:



In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
cd /Users/brendan/Documents/projects/mpa_project/gfw_research

/Users/brendan/Documents/projects/mpa_project/gfw_research


In [4]:
import numpy as np
import pandas
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pywdpa
import geopandas
import contextily as ctx
from shapely import geometry
from shapely import ops
import pretty_html_table
import cProfile

import util

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

/opt/homebrew/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
DATA_PATH = "/Users/brendan/Documents/projects/mpa_project/gfw_research/data/"
FILENAME = "mpatlas_20201223_clean/mpatlas_20201223_clean.shp"

#mpas = geopandas.read_file(DATA_PATH + FILENAME)
# util.load_mpatlas_mpas()
mpas = geopandas.read_parquet("data/mpas/mpatlas.parquet")

In [8]:
mpas

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20933 entries, 0 to 20932
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   mpa_id      20933 non-null  int64   
 1   wdpa_id     20933 non-null  int64   
 2   wdpa_pid    18866 non-null  object  
 3   name        20933 non-null  object  
 4   country     20933 non-null  object  
 5   sovereign   20933 non-null  object  
 6   designatio  20846 non-null  object  
 7   designat_1  20535 non-null  object  
 8   designat_2  20467 non-null  object  
 9   iucn_categ  18911 non-null  object  
 10  status      20933 non-null  object  
 11  status_yea  20933 non-null  int64   
 12  is_mpa      20933 non-null  int64   
 13  implemente  20933 non-null  int64   
 14  implementa  322 non-null    object  
 15  no_take     20933 non-null  object  
 16  no_take_ar  20933 non-null  float64 
 17  rep_m_area  20933 non-null  float64 
 18  calc_m_are  20933 non-null  float64 
 

In [6]:
def load_year(year):
    year = str(year)
    print(f'\r reading year {year}', end='')
    return pandas.read_parquet("data/points/" + year + ".parquet")

In [7]:
points_by_year = {
    year : load_year(year)
    for year in range(2012, 2021)
}

 reading year 2020

In [9]:
# sample the points here before converting to geo
# use the sampled points to find the mmsi that are interesting
# only then should we get all of the points of relevent mmsi

SAMPLE_RATIO = 100
points_sampled = []
for year, points in points_by_year.items():
    print(f'\rsampling {str(year)}... ', end='')
    points_sampled.append(points.sample(frac=1/SAMPLE_RATIO))
print('done.')
points_sampled = pandas.concat(points_sampled)
print("converting to geo...  ", end='')
geopoints_sampled = util.convert_to_geo(points_sampled, box=True)
print("done.")


sampling 2020... done.
converting to geo...  done.


In [9]:
mpas['geography_area'] = (mpas.to_crs(epsg=6933).area / 1000000).astype('int32')



In [13]:
df = pandas.DataFrame(mpas.drop(columns='geometry'))

In [14]:
df.to_csv(path_or_buf='with_geography_area.csv')

In [47]:
int(mpas[mpas['wdpa_id'] == 555512002].to_crs(epsg=6933).area / 1000000)

408397

In [51]:
mpas[mpas['wdpa_id'] == 555512002]

,mpa_id,wdpa_id,wdpa_pid,name,country,sovereign,designatio,designat_1,designat_2,iucn_categ,status,status_yea,is_mpa,implemente,implementa,no_take,no_take_ar,rep_m_area,calc_m_are,rep_area,calc_area,gov_type,mgmt_auth,mgmt_plan_,fishing,access,constancy,permanence,SHAPE_Leng,SHAPE_Area,geometry,geography_area
10124,7705427,555512002,555512002,Phoenix Islands Protected Area,KIR,KIR,World Heritage Site (natural or mixed),World Heritage Site (natural or mixed),International,Not Applicable,Inscribed,2010,0,1,None,All,408224.5,0.0,408145.0,408250.0,408258.0,Not Reported,Not Reported,Not Reported,Some Restrictions,Yes,Year-round,Permanent,22.701703,33.262626,"POLYGON ((-175.31500 -6.61600, -175.86600 -6.6...",408397


In [52]:
no_take = mpas[(mpas['no_take']=='All') & (mpas['implemente'])]
of_interest_frame = no_take[
    ((no_take['status_yea'] > 2012) | (no_take['implementa'] > '2012-01-01')) & no_take['implemente']]
of_interest = [
    (None, 555512002, 'Phoenix Island Protected Area', '2015-01-01', 408145.0, 408258.0)
]
# for i, row in no_take[no_take['status_yea'] > 2012].iterrows():
for i, row in of_interest_frame.iterrows():
    date = row['implementa']
    if date is None:
        date = str(row['status_yea'])
    of_interest.append(
        (row['mpa_id'], row['wdpa_id'], row['name'], date, row['rep_m_area'], row['calc_m_are'])
    )

np.random.shuffle(of_interest)

In [53]:
import util
folder_prefix = f'plots/{np.datetime64("now")}'

plot=False

tables = pandas.DataFrame()
summed_tables = pandas.DataFrame()
j = 0
for mpa_id, wdpa_id, name, date, rep_m_area, calc_m_are in of_interest:
    j += 1
    print(f'\r{j}/{len(of_interest)} {mpa_id} {wdpa_id} {name} {date}                           ', end='')
    mpa = None
    if wdpa_id:
        mpa = mpas[mpas['wdpa_id'] == wdpa_id].dissolve(by='wdpa_id')
    elif mpa_id:
        mpa = mpas[mpas['mpa_id'] == mpa_id].dissolve(by='mpa_id')
    else:
        print(f'no id for {name}, skipping')
        continue
        
    folder = folder_prefix + f'/mpas/{name}/'
    
    returned = util.analyze_mpa(
        geopoints_sampled, points_by_year, mpa, date, name, verbose=False,
        plot=plot, folder=folder)
    if returned is not None:
        table, points_of_mpa_ships = returned

        table.insert(0, 'name', name)
        table.insert(1, 'date', date)
        table.insert(2, 'wdpa_id', wdpa_id)
        table.insert(3, 'mpa_id', mpa_id)
        table.insert(4, 'reported_marine_area', rep_m_area)
        table.insert(5, 'calculated_marine_area', calc_m_are)
        table.insert(6, 'geography_area', mpa['geography_area'].iloc[0])
        

        
        if plot:
            html_table_blue_light = pretty_html_table.build_table(table, 'blue_light')
            # Save to html file
            with open(folder+'individual_ships.html', 'w') as f:
                f.write(html_table_blue_light)
        
        tables = tables.append(table)
                
        summed = table.groupby('name').aggregate({
            'name': 'first',
            'wdpa_id': 'first',
            'mpa_id': 'first',
            'date': 'first',
            'calculated_marine_area': 'first',
            'reported_marine_area': 'first',
            'geography_area': 'first',
            'mmsi': 'count',
            'in_pre': 'sum',
            'out_pre': 'sum',
            'in_post': 'sum',
            'out_post': 'sum'})
        summed_tables = summed_tables.append(summed)
    else:
        summed_tables = summed_tables.append(
            {'name': mpa.iloc[0]['name'], 'wdpa_id': wdpa_id, 'mpa_id': mpa_id, 'date': date}
            , ignore_index=True)

# Populate the summed tables percentages columns
summed_tables['in_pre_p'] = summed_tables['in_pre'] / (summed_tables['in_pre'] + summed_tables['out_pre'])
summed_tables['in_post_p'] = summed_tables['in_post'] / (summed_tables['in_post'] + summed_tables['out_post'])
summed_tables['decrease'] = (summed_tables['in_pre_p'] - summed_tables['in_post_p'])/(summed_tables['in_pre_p'])
summed_tables = summed_tables.fillna(0)


# if plot:
#     # Save to html files
#     html_table = pretty_html_table.build_table(summed_tables.sort_values('decrease', ascending=False), 'blue_light')
#     with open(folder_prefix+'/summed_effort_by_decrease.html', 'w') as f:
#         f.write(html_table)


#     html_table = pretty_html_table.build_table(summed_tables.sort_values('mmsi', ascending=False), 'blue_light')
#     with open(folder_prefix+'/summed_effort_by_mmsi.html', 'w') as f:
#         f.write(html_table)

127/127 68812527 555587198 Two Foot Bay Sanctuary 2014                                         2-01                                               

In [55]:
summed_tables.sort_values('mmsi', ascending=False).to_csv(path_or_buf='of_interest.csv')

In [54]:
summed_tables.sort_values('mmsi', ascending=False)

,name,wdpa_id,mpa_id,date,calculated_marine_area,reported_marine_area,geography_area,mmsi,in_pre,out_pre,in_post,out_post,in_pre_p,in_post_p,decrease
71,Phoenix Island Protected Area,555512002,0,2015-01-01,408258.000000,408145.00,408397.0,127.0,134117.9046,1.399682e+06,816.3618,2.723075e+06,0.087442,0.000300,0.996573
91,Palau,555622118,68807606,2020-01-01,503521.000000,502538.00,105018.0,82.0,256131.6114,3.075875e+05,24039.2755,1.090456e+05,0.454360,0.180631,0.602450
122,Ross Sea Region [General Protection Zone (ii) ...,555624810,68813307,2017-12-01,74472.000000,74765.00,21102.0,21.0,13532.5229,3.057624e+05,2454.3765,1.612849e+05,0.042383,0.014990,0.646327
9,Ross Sea Region [General Protection Zone (i) R...,555624810,9047,2017-12-01,973704.000000,1555851.00,21102.0,21.0,13532.5229,3.057624e+05,2454.3765,1.612849e+05,0.042383,0.014990,0.646327
46,Ross Sea Region [General Protection Zone (iii)...,555624810,68813308,2017-12-01,22643.000000,21098.00,21102.0,21.0,13532.5229,3.057624e+05,2454.3765,1.612849e+05,0.042383,0.014990,0.646327
2,Coral Sea - National Park Zone,555556875,68808920,2018-07-01,239755.000000,238399.00,66480.0,16.0,27370.0224,8.086899e+04,14533.4056,4.822219e+04,0.252867,0.231587,0.084152
109,Terres Australes Françaises [Strict Marine Pro...,345888,68808359,2016-12-16,120378.000000,120250.00,988520.0,12.0,79932.2690,4.948200e+03,56491.7869,6.797991e+03,0.941704,0.892589,0.052155
33,Central Eastern - National Park Zone,555556874,14986,2018-07-01,8135.000000,8109.00,608.0,11.0,5113.6616,7.450151e+04,3085.8355,4.706286e+04,0.064230,0.061534,0.041975
29,Revillagigedo - Zona Núcleo/Uso Restringido Tu...,555629385,68817262,2018-11-30,0.000000,402.69,406.0,9.0,1490.9927,5.654996e+04,0.3849,2.740726e+04,0.025689,0.000014,0.999453
25,Revillagigedo - Zona Núcleo/Uso Restringido In...,555629385,68813451,2018-11-30,148628.000000,147529.00,406.0,9.0,1490.9927,5.654996e+04,0.3849,2.740726e+04,0.025689,0.000014,0.999453
